# Efficiency/Effectiveness Trade-offs in Learning to Rank
### Tutorial @ ICTIR 2017, HandsOn Session N. 1

##### Claudio Lucchese (UniVe), Franco Maria Nardini (ISTI-CNR)
##### High Performance Computing Lab. http://hpc.isti.cnr.it/

<img src="images/hpc.png" width="250">

In [59]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

from rankeval.dataset import Dataset
from rankeval.model import RTEnsemble
from rankeval.analysis.effectiveness import tree_wise_performance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Agenda

Given a trained LambdaMART model (stored in QuickRank format):

0. setup of an experimental environment for testing different scoring methods: Conditional Operators (CondOp), VPred, QuickScorer (QS), Vectorized QuickScorer (v-QS).
0. Execution of the different methods and comparison.
0. Low-level analysis with ``perf`` for two of them: VPred vs QuickScorer
0. Comparison with previously published results [QS-TOIS16].

** Bonus Track **

0. Multi-threaded implementation of Vectorized QuickScorer
0. Multi-threaded scoring with RankEval

### Step 0

Clone and compile QuickRank. Detailed instructions on how to do it can be found at: http://quickrank.isti.cnr.it

### Step 0.1

Clone and install RankEval. Detailed instructions on how to do it can be found at: http://rankeval.isti.cnr.it

### Step 0.2

Download the Istella-S LETOR dataset (http://blog.istella.it/istella-learning-to-rank-dataset/)

In [128]:
# Global Options

# paths to executable files
QUICKRANK      = "./quickrank/bin/quicklearn"
SCORER         = "./quickrank/bin/quickscore"
QUICKSCORER    = "./QuickScorer/bin/quickscorer"
QUICKSCORER_NS = "./QuickScorer-noscoring/bin/quickscorer"
VPRED          = "./asadi_tkde13/out/VPred"
VPRED_NS       = "./asadi_tkde13-noscoring/out/VPred"
PERF           = "perf"

# paths to Istella-S dataset
train_dataset_file       = "/data/letor-datasets/tiscali/sample/train.txt"
valid_dataset_file       = "/data/letor-datasets/tiscali/sample/vali.txt"
test_dataset_file        = "/data/letor-datasets/tiscali/sample/test.txt"

dataset_size = 681250

# The first row of the test file used by VPred should be: "<# rows of the file> <# features>\n".
vpred_test_dataset_file  = "/data/letor-datasets/tiscali/sample/test.vpred"

# paths to model file
models_folder            = "models"
baseline_model_file      = os.path.join(models_folder, "istella-small.lamdamart.xml")

# setting floating point precision of Pandas
pd.set_option('precision', 1)

### Step 1

Load an existing LambdaMART model with RankEval or train it with QuickRank.

In [80]:
# load a QuickRank model
# if no model is available, use the box below to train one!

baseline_model = RTEnsemble(baseline_model_file, name="Baseline", format="QuickRank")

In [4]:
# The code below trains a LambdaMART of 1000 trees.

!{QUICKRANK} \
  --algo LAMBDAMART \
  --num-trees 1000 \
  --shrinkage 0.05 \
  --num-thresholds 0 \
  --num-leaves 64 \
  --min-leaf-support 1 \
  --end-after-rounds 0 \
  --partial 1000 \
  --train {train_dataset_file} \
  --valid {valid_dataset_file} \
  --train-metric NDCG \
  --train-cutoff 10 \
  --model-out ~/quickrank.1000T.64L.xml


      _____  _____
     /    / /____/
    /____\ /    \        QuickRank has been developed by hpc.isti.cnr.it
    ::Quick:Rank::                             mail: quickrank@isti.cnr.it


# Ranker: LAMBDAMART
# max no. of trees = 1000
# no. of tree leaves = 64
# shrinkage = 0.050000
# min leaf support = 1
# no. of thresholds = unlimited

# Reading training dataset: /data/letor-datasets/tiscali/sample/train.txt
#	 Reading time: 95.05 s. @ 31.69 MB/s  (post-proc.: 1.18 s.)
#	 Dataset size: 2043304 x 220 (instances x features)
#	 Num queries: 19245 | Avg. len: 106.173

# Reading validation dataset: /data/letor-datasets/tiscali/sample/vali.txt
#	 Reading time: 32.11 s. @ 31.46 MB/s  (post-proc.: 0.31 s.)
#	 Dataset size: 684076 x 220 (instances x features)
#	 Num queries: 7211 | Avg. len: 94.866

#
# Ranker: LAMBDAMART
# max no. of trees = 1000
# no. of tree leaves = 64
# shrinkage = 0.050
# min leaf support = 1
# no. of thresholds = unlimited
#
# training scorer: NDCG@10
# Initialization

    255   0.7646   0.7380
    256   0.7648   0.7381
    257   0.7649   0.7383 *
    258   0.7650   0.7382
    259   0.7652   0.7384 *
    260   0.7653   0.7384
    261   0.7654   0.7387 *
    262   0.7655   0.7388 *
    263   0.7656   0.7388
    264   0.7658   0.7389 *
    265   0.7659   0.7388
    266   0.7660   0.7388
    267   0.7663   0.7393 *
    268   0.7664   0.7394 *
    269   0.7664   0.7394
    270   0.7665   0.7393
    271   0.7666   0.7395 *
    272   0.7667   0.7396 *
    273   0.7668   0.7396 *
    274   0.7669   0.7398 *
    275   0.7670   0.7399 *
    276   0.7672   0.7400 *
    277   0.7673   0.7402 *
    278   0.7674   0.7404 *
    279   0.7675   0.7404 *
    280   0.7676   0.7404 *
    281   0.7677   0.7403
    282   0.7677   0.7403
    283   0.7678   0.7405 *
    284   0.7679   0.7406 *
    285   0.7682   0.7407 *
    286   0.7683   0.7406
    287   0.7684   0.7406
    288   0.7685   0.7408 *
    289   0.7686   0.7409 *
    290   0.7687   0.7411 *
    291   0.7687  

    557   0.7881   0.7528
    558   0.7882   0.7528
    559   0.7882   0.7528
    560   0.7882   0.7527
    561   0.7883   0.7528
    562   0.7883   0.7528
    563   0.7884   0.7527
    564   0.7884   0.7527
    565   0.7885   0.7528
    566   0.7886   0.7528
    567   0.7887   0.7529
    568   0.7888   0.7529
    569   0.7889   0.7530 *
    570   0.7889   0.7530 *
    571   0.7890   0.7530 *
    572   0.7889   0.7530 *
    573   0.7890   0.7530
    574   0.7890   0.7530
    575   0.7890   0.7532 *
    576   0.7891   0.7531
    577   0.7891   0.7532
    578   0.7892   0.7531
    579   0.7892   0.7531
    580   0.7893   0.7533 *
    581   0.7893   0.7534 *
    582   0.7893   0.7533
    583   0.7894   0.7534 *
    584   0.7894   0.7533
    585   0.7895   0.7533
    586   0.7896   0.7533
    587   0.7896   0.7534 *
    588   0.7897   0.7535 *
    589   0.7897   0.7535 *
    590   0.7898   0.7535
    591   0.7898   0.7536 *
    592   0.7899   0.7535
    593   0.7899   0.7535
    594   0.78

    865   0.8008   0.7574
    866   0.8009   0.7574
    867   0.8009   0.7574
    868   0.8009   0.7574
    869   0.8009   0.7575
    870   0.8010   0.7576
    871   0.8009   0.7575
    872   0.8010   0.7575
    873   0.8011   0.7575
    874   0.8011   0.7575
    875   0.8012   0.7575
    876   0.8012   0.7575
    877   0.8013   0.7576
    878   0.8013   0.7576
    879   0.8013   0.7575
    880   0.8014   0.7575
    881   0.8014   0.7575
    882   0.8014   0.7575
    883   0.8014   0.7576 *
    884   0.8015   0.7576
    885   0.8015   0.7576
    886   0.8016   0.7576
    887   0.8016   0.7575
    888   0.8016   0.7577 *
    889   0.8016   0.7576
    890   0.8017   0.7576
    891   0.8018   0.7576
    892   0.8018   0.7576
    893   0.8018   0.7576
    894   0.8018   0.7576
    895   0.8019   0.7577 *
    896   0.8019   0.7577 *
    897   0.8020   0.7577 *
    898   0.8021   0.7578 *
    899   0.8021   0.7578
    900   0.8022   0.7578 *
    901   0.8022   0.7578
    902   0.8022   0.757

### Step 2

We now translate the LambdaMART model in C++ code employing Conditional Operators to build the final document ranker [QS-TOIS16].

QuickRank provides a plugin to convert models stored in its native XML format to C++ source code. The result is that each tree is translated as a nested block of Conditional Operators (https://www.tutorialspoint.com/cplusplus/cpp_conditional_operator.htm). The obtained C++ code can be compiled to produce a working ranked of the given model.

Here a toy example of a tree:
~~~~
<feature>194</feature>
<threshold>140</threshold>
 <split pos="left">
   <feature>31</feature>
   <threshold>0.0120639997</threshold>
     <split pos="left">
       <output>-0.78920207999267233</output>
     </split>
     <split pos="right">
       <output>1.1050481952095461</output>
     </split>
 </split>
~~~~

The conditional operator (BOOLEAN CONDITION ? THEN : ELSE) translation produces:

~~~~
v[194] <= 140.0f ? ( v[31] <= 0.0120639997f ? -0.78920207999267233 : 1.1050481952095461 )
~~~~

In [107]:
# We use CondOp-based C code as a baseline for the scoring time evaluation

def run_condop(model_file, dataset_file, rounds=1):
    # create the C code
    print (" 1. Creating the C code for " + model_file)
    condop_source = model_file + ".c"
    
    _ = !{QUICKRANK} \
      --generator condop \
      --model-file {model_file} \
      --code-file {condop_source}
    
    # Compile an executable ranker. The resulting ranker is SCORER=./quickrank/bin/quickscore
    print (" 2. Compiling the model")

    # replace empty scorer
    !cp {condop_source} ./quickrank/src/scoring/ranker.cc
    # compile
    _ = !make -j -C ./quickrank/build_ quickscore 
    
    # Now running the Conditional Operators scorer by executing the previously compiled C code.
    # QuickScore options:
    #  -h,--help                             print help message
    #  -d,--dataset <arg>                    Input dataset in SVML format
    #  -r,--rounds <arg> (10)                Number of test repetitions
    #  -s,--scores <arg>                     File where scores are saved (Optional).
    print (" 3. Running the compiled model")
    cond_op_scorer_out = !{SCORER} \
      -d {dataset_file} \
      -r {rounds}
    
    print (cond_op_scorer_out.n)
    
    # takes the scoring time in milli-seconds
    cond_op_scoring_time = float(cond_op_scorer_out.l[-1].split()[-2])* 10**6
    
    return cond_op_scoring_time

In [108]:
condop_efficiency = run_condop(baseline_model_file, test_dataset_file) 

 1. Creating the C code for models/istella-small.lamdamart.xml
 2. Compiling the model
 3. Running the compiled model

      _____  _____
     /    / /____/
    /____\ /    \          QuickRank has been developed by hpc.isti.cnr.it
    ::Quick:Rank::                                   quickrank@isti.cnr.it

#	 Dataset size: 681250 x 220 (instances x features)
#	 Num queries: 6562 | Avg. len: 104
       Total scoring time: 73.6 s.
Avg. Dataset scoring time: 73.6 s.
Avg.    Doc. scoring time: 0.000108 s.


In [109]:
# Store current results
results = pd.DataFrame(columns=['Model', '# Trees', 'Scoring Time µs.'])

results.loc[len(results)] = ['CondOp', baseline_model.n_trees, condop_efficiency]
results

,Model,# Trees,Scoring Time µs.
0,CondOp,1492,108.0


### Step 3

Now scoring with VPred [VPRED].

First of all, we need to convert the QuickRank XML model in the VPRED format. Finally, use it to score the test file.

QuickRank provides a plugin to convert models stored in its native XML format to the textual representation employed by the original VPRED code by Nima Asadi et al. [VPRED]. The plugin outputs a textual file. 
 
The original VPred code and instructions on how to compile, install and use it are available here: https://github.com/lintool/OptTrees

In [110]:
vpred_source = baseline_model_file + ".vpred"

!{QUICKRANK} \
  --generator vpred \
  --model-file {baseline_model_file} \
  --code-file {vpred_source}


      _____  _____
     /    / /____/
    /____\ /    \        QuickRank has been developed by hpc.isti.cnr.it
    ::Quick:Rank::                             mail: quickrank@isti.cnr.it

generating VPred input file from: models/istella-small.lamdamart.xml


In [111]:
# Now running the VPred scorer by using the previously converted code.
# note that we are using the original VPred code by Asadi et al. [VPRED].
# The code is available here: https://github.com/lintool/OptTrees

vpred_scorer_out = !{VPRED} \
  -ensemble {vpred_source} \
  -instances {vpred_test_dataset_file} \
  -maxLeaves 64
    
print (vpred_scorer_out.n)

# takes the scoring time in milli-seconds
vpred_scoring_time = float(vpred_scorer_out.l[0].split('\t')[1])* 10**6

$	0.000102213185
Ignore this number: -2779350


In [112]:
# Store current results
results.loc[len(results)] = ['VPred', baseline_model.n_trees, vpred_scoring_time]
results

,Model,# Trees,Scoring Time µs.
0,CondOp,1492,108.0
1,VPred,1492,102.2


### Step 4

QuickScorer uses a novel traversal methods and a cache-friendly data layout that reduces dramatically the traversal time [QS-SIGIR15, QS-TOIS16].

In [113]:
# Now running QuickScorer.
# note that we are using the original QuickScorer code by Lucchese et al. [QS-SIGIR15,QS-TOIS16].
# The code is available under NDA.
#
# Options:
#  -h [ --help ]                     Print help messages.
#  -d [ --dataset ] arg              Path of the dataset to score (SVML format).
#  -r [ --rounds ] arg (=10)         Number of test repetitions.
#  -s [ --scores ] arg               Path of the file where final scores are
#                                    saved.
#  -t [ --tree_type ] arg (=0)       Specify the type of the tree in the
#                                    ensemble:
#                                     - 0 for normal trees,
#                                     - 1 for oblivious trees,
#                                     - 2 for normal trees (reversed blocked),
#                                     - 3 for normal trees (SIMD: SSE/AVX).
#  -m [ --model ] arg                Path of the XML file storing the model.
#  -l [ --nleaves ] arg              Maximum number of leaves in a tree (<= 64).
#  --avx                             Use AVX 256 instructions (at least 8 doc
#                                    blocking).
#  --omp                             Use OpenMP multi-threading document scoring
#                                    (only SIMD: SSE/AVX).

qs_scorer_out = !{QUICKSCORER} \
  -d {test_dataset_file} \
  -m {baseline_model_file} \
  -l 64 \
  -r 1 \
  -t 0
    
print (qs_scorer_out.n)
    
# takes the scoring time in milli-seconds
qs_scoring_time = float(qs_scorer_out.l[-1].split()[-2])* 10**6


      _____  _____
     /    / /____
    /____\ _____/          QuickScorer has been developed by hpc.isti.cnr.it
    :Quick:Scorer:                                   quickscorer@isti.cnr.it

#	 Dataset size: 681250 x 220 (instances x features)
#	 Num queries: 6562 | Avg. len: 104
       Total scoring time: 15.7592 s.
Avg. Dataset scoring time: 15.7592 s.
Avg.    Doc. scoring time: 2.31327e-05 s.


In [114]:
# Store current results
results.loc[len(results)] = ['QS', baseline_model.n_trees, qs_scoring_time]
results

,Model,# Trees,Scoring Time µs.
0,CondOp,1492,108.0
1,VPred,1492,102.2
2,QS,1492,23.1


### Step 5

Vectorized QuickScorer improves over QuickScorer by exploiting 256-bits wide CPU registers [QS-SIGIR16].


In [115]:
# Now running Vectorized QuickScorer (AVX2)
# note that we are using the original QuickScorer code by Lucchese et al. [QS-SIGIR16].
# The code is available under NDA.
#
# Options:
#  -h [ --help ]                     Print help messages.
#  -d [ --dataset ] arg              Path of the dataset to score (SVML format).
#  -r [ --rounds ] arg (=10)         Number of test repetitions.
#  -s [ --scores ] arg               Path of the file where final scores are
#                                    saved.
#  -t [ --tree_type ] arg (=0)       Specify the type of the tree in the
#                                    ensemble:
#                                     - 0 for normal trees,
#                                     - 1 for oblivious trees,
#                                     - 2 for normal trees (reversed blocked),
#                                     - 3 for normal trees (SIMD: SSE/AVX).
#  -m [ --model ] arg                Path of the XML file storing the model.
#  -l [ --nleaves ] arg              Maximum number of leaves in a tree (<= 64).
#  -v [ --doc_block_size ] arg (=1)  Document block size (allowed values:
#                                    1,2,4,8,16; 1 means no blocking).
#  --avx                             Use AVX 256 instructions (at least 8 doc
#                                    blocking).
#  --omp                             Use OpenMP multi-threading document scoring
#                                    (only SIMD: SSE/AVX).

vqs_scorer_out = !{QUICKSCORER} \
  -d {test_dataset_file} \
  -m {baseline_model_file} \
  -l 64 \
  -r 1 \
  -t 3 \
  -v 8 \
  --avx
    
print (vqs_scorer_out.n)
    
# takes the scoring time in milli-seconds
vqs_scoring_time = float(vqs_scorer_out.l[-1].split()[-2])* 10**6


      _____  _____
     /    / /____
    /____\ _____/          QuickScorer has been developed by hpc.isti.cnr.it
    :Quick:Scorer:                                   quickscorer@isti.cnr.it

#	 Dataset size: 681250 x 220 (instances x features)
#	 Num queries: 6562 | Avg. len: 104
       Total scoring time: 10.4056 s.
Avg. Dataset scoring time: 10.4056 s.
Avg.    Doc. scoring time: 1.52742e-05 s.


In [116]:
# Store current results
results.loc[len(results)] = ['v-QS', baseline_model.n_trees, vqs_scoring_time]
results

,Model,# Trees,Scoring Time µs.
0,CondOp,1492,108.0
1,VPred,1492,102.2
2,QS,1492,23.1
3,v-QS,1492,15.3


from [QS-TOIS16]

Some considerations:

0. We reproduce the evaluation methodology presented in [QS-SIGIR15, QS-TOIS16] on a different LambdaMART. The LambdaMART here is composed of 1,492 trees while results in the two papers above are for 1,000 or 5,000 trees.
0. The scoring time is 1.5x higher than the results reported in [QS-TOIS16] for all methods. This because we are running these experiments on a slower machine than the one used for producing the experimental results presented in [QS-SIGIR15, QS-TOIS16].

![caption](images/HO1-scoring.png)

### Step 6

Low-level statistics of the scorer with ```perf```

**perf** (https://perf.wiki.kernel.org/index.php/Tutorial) is a profiler tool for Linux 2.6+ based systems that abstracts away CPU hardware differences in Linux performance measurements and presents a simple commandline interface.

### Step 6.1

```perf``` on QuickScorer

In [117]:
# Below, perf is used to monitor several behaviours of the scorer:
# - L1 cache performance (references and misses): L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses
# - L3 cache performance (references and misses): cache-references,cache-misses
# - number of instructions and cycles: instructions,cycles
# - total number of branches and branch misprediction: branches,branch-misses

perf_out = !{PERF} stat -e \
  L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,\
L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses,\
instructions,cycles,cache-references,cache-misses,branches,branch-misses\
    {QUICKSCORER} \
      -d {test_dataset_file} \
      -m {baseline_model_file} \
      -l 64 \
      -r 1 \
      -t 0

print (perf_out.n)


      _____  _____
     /    / /____
    /____\ _____/          QuickScorer has been developed by hpc.isti.cnr.it
    :Quick:Scorer:                                   quickscorer@isti.cnr.it

#	 Dataset size: 681250 x 220 (instances x features)
#	 Num queries: 6562 | Avg. len: 104
       Total scoring time: 15.8269 s.
Avg. Dataset scoring time: 15.8269 s.
Avg.    Doc. scoring time: 2.32322e-05 s.

 Performance counter stats for './QuickScorer/bin/quickscorer -d /data/letor-datasets/tiscali/sample/test.txt -m models/istella-small.lamdamart.xml -l 64 -r 1 -t 0':

   107,441,099,482 L1-dcache-loads                                              [36.36%]
     5,768,612,463 L1-dcache-load-misses     #    5.37% of all L1-dcache hits   [36.36%]
    55,253,754,540 L1-dcache-stores                                             [36.36%]
       631,099,138 L1-dcache-store-misses                                       [36.37%]
   <not supported> L1-icache-loads         
        31,893,046 L1-icache-lo

In [138]:
# parsing perf output
num_istructions = int(perf_out[20].strip().split(' ')[0].replace(',', ''))
num_cache_ref = int(perf_out[22].strip().split(' ')[0].replace(',', ''))
num_cache_miss = int(perf_out[23].strip().split(' ')[0].replace(',', ''))
num_branches = int(perf_out[24].strip().split(' ')[0].replace(',', ''))
num_branch_misses = int(perf_out[25].strip().split(' ')[0].replace(',', ''))

### Step 6.2

```perf``` on QuickScorer (no scoring).

In [118]:
# Below, perf is used to monitor several behaviours of the scorer:
# - L1 cache performance (references and misses): L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses
# - L3 cache performance (references and misses): cache-references,cache-misses
# - number of instructions and cycles: instructions,cycles
# - total number of branches and branch misprediction: branches,branch-misses

perf_noscoring_out = !{PERF} stat -e \
  L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,\
L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses,\
instructions,cycles,cache-references,cache-misses,branches,branch-misses\
    {QUICKSCORER_NS} \
      -d {test_dataset_file} \
      -m {baseline_model_file} \
      -l 64 \
      -r 1 \
      -t 0
        
print (perf_noscoring_out.n)


      _____  _____
     /    / /____
    /____\ _____/          QuickScorer has been developed by hpc.isti.cnr.it
    :Quick:Scorer:                                   quickscorer@isti.cnr.it

#	 Dataset size: 681250 x 220 (instances x features)
#	 Num queries: 6562 | Avg. len: 104
       Total scoring time: 7.7e-08 s.
Avg. Dataset scoring time: 7.7e-08 s.
Avg.    Doc. scoring time: 1.13028e-13 s.

 Performance counter stats for './QuickScorer-noscoring/bin/quickscorer -d /data/letor-datasets/tiscali/sample/test.txt -m models/istella-small.lamdamart.xml -l 64 -r 1 -t 0':

    54,580,005,206 L1-dcache-loads                                              [36.37%]
       172,014,442 L1-dcache-load-misses     #    0.32% of all L1-dcache hits   [36.39%]
    38,995,829,137 L1-dcache-stores                                             [36.40%]
        99,944,662 L1-dcache-store-misses                                       [36.40%]
   <not supported> L1-icache-loads         
        26,027,639 L1

In [139]:
# parsing perf output
num_istructions_ns = int(perf_noscoring_out[20].strip().split(' ')[0].replace(',', ''))
num_cache_ref_ns = int(perf_noscoring_out[22].strip().split(' ')[0].replace(',', ''))
num_cache_miss_ns = int(perf_noscoring_out[23].strip().split(' ')[0].replace(',', ''))
num_branches_ns = int(perf_noscoring_out[24].strip().split(' ')[0].replace(',', ''))
num_branch_misses_ns = int(perf_noscoring_out[25].strip().split(' ')[0].replace(',', ''))

### Step 6.3

now computing differences between the two runs to get the low level statistics for the scoring part of QS.

In [140]:
# Store current results
perf_results = pd.DataFrame(columns=['Method', 'Instructions', 'Cache Misses', 'Branch Misprediction'])

normalized_instruction_count = (num_istructions - num_istructions_ns) / float(dataset_size * baseline_model.n_trees)
normalized_cache_miss = (num_cache_miss - num_cache_miss_ns) / float(dataset_size * baseline_model.n_trees)
normalized_branch_miss = (num_branch_misses - num_branch_misses_ns) / float(dataset_size * baseline_model.n_trees)

perf_results.loc[len(perf_results)] = ['QS',
                                  normalized_instruction_count,
                                  normalized_cache_miss,
                                  normalized_branch_miss]
perf_results

,Method,Instructions,Cache Misses,Branch Misprediction
0,QS,77.1,1.8e-03,0.1


### Step 6.4

The same methodology now on VPred

In [126]:
# Below, perf is used to monitor several behaviours of the scorer:
# - L1 cache performance (references and misses): L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses
# - L3 cache performance (references and misses): cache-references,cache-misses
# - number of instructions and cycles: instructions,cycles
# - total number of branches and branch misprediction: branches,branch-misses

vpred_perf_out = !{PERF} stat -e \
  L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,\
L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses,\
instructions,cycles,cache-references,cache-misses,branches,branch-misses\
    {VPRED} \
      -ensemble {vpred_source} \
      -instances {vpred_test_dataset_file} \
      -maxLeaves 64
        
print (vpred_perf_out.n)

$	0.0001024996
Ignore this number: -2779350

 Performance counter stats for './asadi_tkde13/out/VPred -ensemble models/istella-small.lamdamart.xml.vpred -instances /data/letor-datasets/tiscali/sample/test.vpred -maxLeaves 64':

   265,219,690,462 L1-dcache-loads                                              [36.36%]
     1,396,690,892 L1-dcache-load-misses     #    0.53% of all L1-dcache hits   [36.36%]
    89,900,989,195 L1-dcache-stores                                             [36.36%]
        24,883,290 L1-dcache-store-misses                                       [36.36%]
   <not supported> L1-icache-loads         
     9,354,194,157 L1-icache-load-misses     #    0.00% of all L1-icache hits   [36.37%]
   625,127,341,692 instructions              #    2.09  insns per cycle         [45.46%]
   298,491,704,130 cycles                    [45.46%]
    15,796,189,517 cache-references                                             [45.46%]
        10,809,889 cache-misses              #    0

In [141]:
# parsing perf output
num_istructions = int(vpred_perf_out[11].strip().split(' ')[0].replace(',', ''))
num_cache_ref = int(vpred_perf_out[13].strip().split(' ')[0].replace(',', ''))
num_cache_miss = int(vpred_perf_out[14].strip().split(' ')[0].replace(',', ''))
num_branches = int(vpred_perf_out[15].strip().split(' ')[0].replace(',', ''))
num_branch_misses = int(vpred_perf_out[16].strip().split(' ')[0].replace(',', ''))

### Step 6.5

``perf`` on VPred (no scoring).

In [129]:
# Below, perf is used to monitor several behaviours of the scorer:
# - L1 cache performance (references and misses): L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses
# - L3 cache performance (references and misses): cache-references,cache-misses
# - number of instructions and cycles: instructions,cycles
# - total number of branches and branch misprediction: branches,branch-misses

vpred_perf_noscoring_out = !{PERF} stat -e \
  L1-dcache-loads,L1-dcache-load-misses,L1-dcache-stores,\
L1-dcache-store-misses,L1-icache-loads,L1-icache-load-misses,\
instructions,cycles,cache-references,cache-misses,branches,branch-misses\
    {VPRED_NS} \
      -ensemble {vpred_source} \
      -instances {vpred_test_dataset_file} \
      -maxLeaves 64
        
print (vpred_perf_noscoring_out.n)

$	1.35045872e-13
Ignore this number: 0

 Performance counter stats for './asadi_tkde13-noscoring/out/VPred -ensemble models/istella-small.lamdamart.xml.vpred -instances /data/letor-datasets/tiscali/sample/test.vpred -maxLeaves 64':

    43,924,987,417 L1-dcache-loads                                              [36.34%]
        39,034,171 L1-dcache-load-misses     #    0.09% of all L1-dcache hits   [36.35%]
    31,210,751,758 L1-dcache-stores                                             [36.40%]
        19,913,536 L1-dcache-store-misses                                       [36.41%]
   <not supported> L1-icache-loads         
         3,333,337 L1-icache-load-misses     #    0.00% of all L1-icache hits   [36.40%]
   175,364,020,154 instructions              #    2.32  insns per cycle         [45.49%]
    75,705,977,800 cycles                    [45.48%]
        21,001,104 cache-references                                             [45.48%]
         7,073,744 cache-misses              #

In [142]:
# parsing perf output
num_istructions_ns = int(vpred_perf_noscoring_out[11].strip().split(' ')[0].replace(',', ''))
num_cache_ref_ns = int(vpred_perf_noscoring_out[13].strip().split(' ')[0].replace(',', ''))
num_cache_miss_ns = int(vpred_perf_noscoring_out[14].strip().split(' ')[0].replace(',', ''))
num_branches_ns = int(vpred_perf_noscoring_out[15].strip().split(' ')[0].replace(',', ''))
num_branch_misses_ns = int(vpred_perf_noscoring_out[16].strip().split(' ')[0].replace(',', ''))

### Step 6.6

now computing differences between the two runs to get the low level statistics for the scoring part of VPred.

In [143]:
# Store current results
normalized_instruction_count = (num_istructions - num_istructions_ns) / float(dataset_size * baseline_model.n_trees)
normalized_cache_miss = (num_cache_miss - num_cache_miss_ns) / float(dataset_size * baseline_model.n_trees)
normalized_branch_miss = (num_branch_misses - num_branch_misses_ns) / float(dataset_size * baseline_model.n_trees)

perf_results.loc[len(perf_results)] = ['VPred',
                                  normalized_instruction_count,
                                  normalized_cache_miss,
                                  normalized_branch_miss]
perf_results

,Method,Instructions,Cache Misses,Branch Misprediction
0,QS,77.1,1.8e-03,1.2e-01
1,VPred,442.5,3.7e-03,4.3e-02


### Step 6.7

from [QS-TOIS16]

Some considerations:

0. We reproduce the methodology presented in [QS-SIGIR15, QS-TOIS16] on a different LambdaMART. The LambdaMART here is composed of 1,492 trees while results in the two papers above are for 1,000 or 5,000 trees. Given that said, the low level behavior of the two methods is confirmed.
0. The number of instructions executed by VPred is the largest one. This is because VPred always runs ``d`` steps, where ``d`` is the depth of a tree even if a document might reach an exit leaf earlier. On the other hand, QS executes the smallest number instructions. This is due to the different traversal strategy of the ensemble, as QS needs to process the false nodes only.
0. In terms of number of branches, we note that QS has a larger total number of branch mispredictions than VPred, which uses scoring functions that are branch-free.
0. In terms of cache misses, we note that QS has a lower cache miss. This is mostly due to the new data layout of QS that perform document scoring by means of linear scans of arrays.

![caption](images/HO1-lowlevelperf.png)

### Bonus Track

### Step 7

We prepare a Multithreaded implementation of Vectorized QuickScorer that exploits OpenMP to distribute bunches of documents to threads scoring them in parallel.

In [144]:
# setting up environment variables for OpenMP
os.environ['OMP_NUM_THREADS']='32'
os.environ['OMP_DISPLAY_ENV']='VERBOSE'
os.environ['OMP_SCHEDULE']='auto'
os.environ['GOMP_CPU_AFFINITY']='0-7,8-15'

In [145]:
# Now running Multi-threaded Vectorized QuickScorer.
# Options:
#  -h [ --help ]                     Print help messages.
#  -d [ --dataset ] arg              Path of the dataset to score (SVML format).
#  -r [ --rounds ] arg (=10)         Number of test repetitions.
#  -s [ --scores ] arg               Path of the file where final scores are
#                                    saved.
#  -t [ --tree_type ] arg (=0)       Specify the type of the tree in the
#                                    ensemble:
#                                     - 0 for normal trees,
#                                     - 1 for oblivious trees,
#                                     - 2 for normal trees (reversed blocked),
#                                     - 3 for normal trees (SIMD: SSE/AVX).
#  -m [ --model ] arg                Path of the XML file storing the model.
#  -l [ --nleaves ] arg              Maximum number of leaves in a tree (<= 64).
#  -v [ --doc_block_size ] arg (=1)  Document block size (allowed values:
#                                    1,2,4,8,16; 1 means no blocking).
#  --avx                             Use AVX 256 instructions (at least 8 doc
#                                    blocking).
#  --omp                             Use OpenMP multi-threading document scoring
#                                    (only SIMD: SSE/AVX).

scorer_out = !{QUICKSCORER} \
  -d {test_dataset_file} \
  -m {baseline_model_file} \
  -l 64 \
  -r 1 \
  -t 3 \
  -v 8 \
  --avx \
  --omp
    
print (scorer_out.n)
    
# takes the scoring time in milli-seconds
scoring_time = float(scorer_out.l[-1].split()[-2])* 10**6


OPENMP DISPLAY ENVIRONMENT BEGIN
  _OPENMP = '201511'
  OMP_DYNAMIC = 'FALSE'
  OMP_NESTED = 'FALSE'
  OMP_NUM_THREADS = '32'
  OMP_SCHEDULE = 'AUTO'
  OMP_PROC_BIND = 'TRUE'
  OMP_PLACES = '{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15}'
  OMP_STACKSIZE = '0'
  OMP_WAIT_POLICY = 'PASSIVE'
  OMP_THREAD_LIMIT = '4294967295'
  OMP_MAX_ACTIVE_LEVELS = '2147483647'
  OMP_CANCELLATION = 'FALSE'
  OMP_DEFAULT_DEVICE = '0'
  OMP_MAX_TASK_PRIORITY = '0'
  GOMP_CPU_AFFINITY = ''
  GOMP_STACKSIZE = '0'
  GOMP_SPINCOUNT = '300000'
OPENMP DISPLAY ENVIRONMENT END

      _____  _____
     /    / /____
    /____\ _____/          QuickScorer has been developed by hpc.isti.cnr.it
    :Quick:Scorer:                                   quickscorer@isti.cnr.it

#	 Dataset size: 681250 x 220 (instances x features)
#	 Num queries: 6562 | Avg. len: 104
       Total scoring time: 0.845968 s.
Avg. Dataset scoring time: 0.845968 s.
Avg.    Doc. scoring time: 1.24179e-06 s.


In [146]:
# Store current results
results.loc[len(results)] = ['vQS-OMP', baseline_model.n_trees, scoring_time]
results

,Model,# Trees,Scoring Time µs.
0,CondOp,1492,108.0
1,VPred,1492,102.2
2,QS,1492,23.1
3,v-QS,1492,15.3
4,vQS-OMP,1492,1.2


### Step 8

RankEval (http://rankeval.isti.cnr.it) - multithread scoring written in Cython

In [147]:
from rankeval.dataset.datasets_fetcher import load_dataset

dataset_container = load_dataset(dataset_name='istella-sample',
                                download_if_missing=True, 
                                force_download=False, 
                                with_models=False)

Loading files. This may take a few minutes.
done loading dataset!


In [148]:
# We now use RankEval to score the test file.
scorer_out = %timeit -o baseline_model.score(dataset_container.test_dataset, False)

The slowest run took 39840.24 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 263 µs per loop


### References

[VPRED] Asadi et al. Runtime Optimizations for Tree-Based Machine Learning Models. IEEE Trans. Knowl. Data Eng. 26(9): 2281-2292 (2014).

[QS-SIGIR15] Lucchese et al. QuickScorer: A Fast Algorithm to Rank Documents with Additive Ensembles of Regression Trees. ACM SIGIR 2015. Best Paper Award.

[QS-TOIS16] Dato et al. Fast Ranking with Additive Ensembles of Oblivious and Non-Oblivious Regression Trees. ACM TOIS, Vol. 9, No. 4. Dec. 2016.

[QS-SIGIR16] Lucchese et al. Exploiting CPU SIMD Extensions to Speed-up Document Scoring with Tree Ensembles. ACM SIGIR 2016.